In [ ]:
!pip install pyformlang

In [ ]:
from pyformlang.finite_automaton import DeterministicFiniteAutomaton, State
from pyformlang.fst import FST
import time
import re
import random
import unittest

In [ ]:
dialogs = {}
descriptions = {}

In [ ]:
t = {
  'q0':
    ('Estás en la ciudad de NeoTech, un mundo futurista donde la tecnología ha alcanzado su máxima expresión. La ciudad está llena de rascacielos brillantes y calles limpias, pero también oculta oscuros secretos bajo su apariencia impecable. Eres el detective Alex Mercer, te encuentras en su oficina, mirando por la ventana la lluvia ácida que cae sobre la ciudad. Mercer es conocido por ser un investigador astuto y despiadado que nunca ha perdido un caso. La noticia de un crimen brutal ha llegado a sus oídos, y está decidido a resolverlo a toda costa.',
    ['Elige investigar el crimen por tu cuenta. Desconfías de las autoridades y prefieres seguir tu propio camino. Tomarás tu abrigo, cogerás tu pistola futurista y te aventurarás en las calles de NeoTech en busca de pistas por tu cuenta.', 'Decides colaborar con la unidad especial de investigación de NeoTech. Confías en la tecnología avanzada de la ciudad y crees que trabajar junto a las autoridades te dará una ventaja en la resolución del caso. Te comunicas con tu contacto en la unidad y te diriges a la sede central.']),
  'q1':
    ('El detective Alex Mercer se pone su abrigo y coge su pistola futurista antes de salir a las peligrosas calles de NeoTech. La lluvia ácida sigue cayendo mientras te sumerges en la oscuridad de la ciudad.',
    ['Te diriges al barrio bajo de NeoTech, donde la mayoría de los criminales se esconden. Allí, interrogas a informantes y buscas pistas en los lugares más sombríos. Esta elección te llevará por un camino lleno de encuentros peligrosos y secretos oscuros. Puedes descubrir información valiosa, pero también enfrentarás amenazas constantes.', 'Optas por investigar en los sectores de alta tecnología de NeoTech. Crees que el crimen podría estar relacionado con avances tecnológicos secretos. Te sumerges en el mundo de las corporaciones y los científicos brillantes, tratando de encontrar conexiones con el crimen. Esta elección te llevará por un camino lleno de intriga tecnológica y conspiraciones. Podrías descubrir avances tecnológicos sorprendentes, pero también te enfrentarás a la resistencia de poderosas entidades corporativas.']),
  'q3':
    ('El detective Alex Mercer se adentra en el sombrío barrio bajo de NeoTech, donde los criminales y desesperados pululan por las calles estrechas y oscuras. Mientras exploras esta parte de la ciudad, te enfrentas a dos caminos:',
    ['Te encuentras con un grupo de informantes callejeros que te proporcionan información sobre el crimen. Descubres que la víctima tenía conexiones con una peligrosa pandilla cibernética llamada "Los Renegados". Puedes decidir infiltrarte en su territorio para obtener más pistas, pero eso conlleva un riesgo considerable.', 'Encuentras una escena de crimen donde la víctima fue asesinada. Examinas el lugar detenidamente y descubres un tatuaje en el cuerpo que podría ser clave para la investigación. Sin embargo, esta acción atrae la atención de Los Renegados, quienes te persiguen por interferir en sus asuntos.']),
  'q7':
    ('Alex Mercer decide infiltrarse en el territorio de "Los Renegados", la peligrosa pandilla cibernética sospechosa de estar relacionada con el crimen. Al hacerlo, se enfrenta a dos opciones:',
    ['Te infiltras en la base de Los Renegados de manera sigilosa. A medida que exploras su guarida subterránea, descubres pistas que vinculan a la víctima con actividades ilegales. Sin embargo, tu presencia no pasa desapercibida y debes enfrentarte a un enfrentamiento con miembros de la pandilla. ¿Lucharás para escapar o intentarás obtener más información antes de huir?', 'Optas por confrontar abiertamente a Los Renegados. Tu valentía impresiona a algunos miembros de la pandilla, lo que te da la oportunidad de negociar. Descubres que la víctima tenía información valiosa sobre un gran atraco planeado por Los Renegados. Ahora debes decidir si te unirás a ellos para descubrir la verdad o si informarás a las autoridades, lo que podría poner en peligro tu vida.']),
  'q15':
    ('Después de infiltrarte en la base de "Los Renegados", descubres pistas que vinculan a la víctima con actividades ilegales. Ahora, enfrentas nuevas decisiones',
    ['Decides luchar contra los miembros de Los Renegados en un intento desesperado de obtener más información. Durante la pelea, encuentras una grabación que revela la participación de un alto funcionario de la ciudad en el crimen. Decides investigarlo más a fondo con la ayuda de un viejo contacto.', 'Optas por escapar de la base de Los Renegados sin luchar. Sales de allí con las pistas que has encontrado, pero sin alertar a la pandilla. Esto te permite continuar tu investigación de manera más discreta, pero también te deja con la incertidumbre de lo que podrías haber descubierto en una confrontación.']),
  'q16':
    ('Después de confrontar abiertamente a Los Renegados, ahora debes tomar decisiones clave:',
    ['Después de ganar la admiración de algunos miembros de Los Renegados, te invitan a unirte a ellos en su gran atraco. Al hacerlo, te sumerges en el mundo criminal y participas en una serie de atracos arriesgados. Tu objetivo es descubrir la verdad detrás del crimen y exponer a los líderes de la pandilla, pero esto podría poner en peligro tu integridad moral.', 'Decides informar a las autoridades sobre la oferta de Los Renegados. La unidad de élite de NeoTech está lista para una redada en el escondite de la pandilla. Como resultado, desmantelan parte de la operación de Los Renegados, pero la pandilla se da cuenta de que eres un traidor. Ahora, te persiguen ferozmente y debes buscar refugio mientras continuas tu investigación.']),
  'q8':
    ('Tras interferir en los asuntos de Los Renegados, te encuentras siendo perseguido por la pandilla. Tienes que tomar decisiones cruciales:',
    ['Optas por escapar de Los Renegados por las estrechas calles del barrio bajo de NeoTech. Debes utilizar tu astucia y conocimiento del terreno para eludir a tus perseguidores. A lo largo del camino, encuentras pistas adicionales que sugieren que la víctima estaba investigando la corrupción en la policía de NeoTech.', 'Decides enfrentarte a Los Renegados en un enfrentamiento a tiros. Esta elección te lleva a un intenso tiroteo en las calles oscuras del barrio bajo. Puedes ganar información vital de algunos de los miembros derrotados, pero también corres el riesgo de quedar gravemente herido.']),
  'q17':
    ('Después de escapar de Los Renegados en las calles oscuras del barrio bajo de NeoTech, Mercer se enfrenta a nuevas decisiones',
    ['Te refugias en un bar clandestino frecuentado por criminales. Mientras estás allí, escuchas conversaciones sobre un misterioso individuo conocido como "El Informante". Decides investigar más a fondo para descubrir quién es y si puede proporcionarte información valiosa sobre el crimen.', 'En lugar de buscar refugio, decides rastrear a los miembros de Los Renegados que te persiguen. Los sigues hasta su escondite secreto, donde descubres evidencia de que están planeando un ataque importante en la ciudad.']),
  'q18':
    ('Después de enfrentarte a Los Renegados en un tiroteo en las calles oscuras del barrio bajo, Mercer se enfrenta a nuevas decisiones:',
    ['Tras derrotar a los miembros de Los Renegados, los interrogas y descubres información importante sobre el líder de la pandilla. Decides buscar pruebas adicionales antes de confrontarlo. La confrontación podría ser peligrosa, pero también puede ser la clave para resolver el crimen.', 'Decides tomar un miembro de Los Renegados como rehén para obtener información adicional. Esto te lleva a una tensa negociación donde debes decidir qué estás dispuesto a ofrecer a cambio de la información que necesitas. Esta elección podría tener consecuencias morales y éticas importantes.']),
  'q4':
    ('Decides explorar los sectores de alta tecnología de NeoTech, donde las corporaciones poderosas y los científicos brillantes dominan el escenario. Aquí, te encuentras con dos opciones:',
    ['Te infiltras en una de las principales corporaciones tecnológicas en busca de pistas. Descubres que la víctima había estado investigando un proyecto altamente confidencial. A medida que profundizas en la investigación, te enfrentas a decisiones éticas complicadas sobre si revelar la verdad o proteger los secretos corporativos.', 'Entablas conversaciones con un científico brillante que estaba colaborando con la víctima en un proyecto secreto. El científico te ofrece información importante, pero también te pide que lo ayudes a escapar de las garras de la corporación, ya que teme por su vida.']),
  'q9':
    ('Dentro de la corporación tecnológica, Mercer se encuentra en una encrucijada:',
    ['Decides revelar la información sobre el proyecto confidencial a las autoridades. Esto provoca una investigación oficial y la corporación está bajo escrutinio público. Tu acción desencadena una serie de eventos que ponen en peligro tu seguridad, ya que la corporación intenta silenciarte para proteger sus secretos.', 'Optas por proteger los secretos corporativos y no revelar la información. Esto te permite obtener acceso a información aún más valiosa sobre el proyecto. Sin embargo, te ves atrapado en un dilema moral sobre si debes priorizar la verdad o tus propios intereses.']),
  'q19':
    ('Después de revelar la información sobre el proyecto confidencial a las autoridades, Mercer se enfrenta a nuevas decisiones:',
    ['La revelación de la información provoca una investigación oficial sobre la corporación tecnológica. Como resultado, algunos ejecutivos de la empresa son arrestados y la corporación está bajo escrutinio público. Esto te convierte en un testigo clave en el caso y te ofrecen protección policial, pero también te convierte en un blanco para aquellos que desean silenciarte. Decides colaborar plenamente con la investigación y reunirte con altos funcionarios gubernamentales.', 'La corporación, al enterarse de que revelaste la información, lanza una campaña de desprestigio en tu contra. Eres acusado de difamación y te enfrentas a una batalla legal agotadora. Decides evitarte los problemas y retirarte del caso']),
  'q20':
    ('Después de optar por proteger los secretos corporativos, Mercer se enfrenta a nuevas decisiones:',
    ['La corporación te considera un valioso aliado y te brinda acceso a información aún más confidencial. Te conviertes en parte de un círculo interno de la corporación y obtienes conocimiento sobre proyectos aún más oscuros. Esto te permite obtener pistas adicionales sobre el crimen, pero también te hace cómplice de las actividades ilícitas de la empresa.', 'A medida que profundizas en la investigación, te das cuenta de la magnitud de la corrupción en la corporación. Decides revelar la información a un periodista de investigación de renombre que ha estado siguiendo la pista del mismo caso. Esto provoca una exposición masiva de la corrupción corporativa y te convierte en un objetivo para los enemigos de la corporación. Ahora debes lidiar con las consecuencias de tu elección y la amenaza constante.']),
  'q10':
    ('El científico que te contactó está agradecido por tu ayuda y te proporciona información vital',
    ['Ayudas al científico a escapar de la corporación. Juntos, se embarcan en una huida peligrosa mientras intentan exponer los oscuros secretos de la corporación. Esto te lleva a una trama de conspiración y persecución mientras luchas por tu supervivencia y la revelación de la verdad.', 'Decides tomar la información del científico y abandonarlo a su suerte. Esto te permite acceder a información valiosa, pero también te atormenta la conciencia por tu falta de compasión. La corporación comienza a sospechar de tus actividades y se convierte en un enemigo aún más formidable.']),
  'q21':
    ('Después de ayudar al científico a escapar de la corporación, Mercer se enfrenta a nuevas decisiones:',
    ['El científico y tú deciden buscar refugio en la resistencia clandestina de NeoTech, un grupo de activistas que luchan contra las corporaciones poderosas. Te unes a su causa y te conviertes en un agente doble que recopila información interna de la corporación para la resistencia. A medida que profundizas en la lucha contra las corporaciones, te encuentras en una posición de riesgo constante.', 'En lugar de unirte a la resistencia, decides mantener una relación independiente con el científico. Juntos, se embarcan en una investigación secreta para exponer los oscuros secretos de la corporación. Esto te lleva por un camino peligroso mientras intentas equilibrar tus propios intereses con la búsqueda de la verdad.']),
  'q22':
    ('Después de tomar la información del científico y abandonarlo, Mercer se enfrenta a nuevas decisiones:',
    ['Utilizas la información proporcionada por el científico para ganar acceso a la sede principal de la corporación. Te conviertes en un infiltrado y comienzas a recopilar pruebas incriminatorias desde adentro. Sin embargo, tu papel en la corporación te pone en constante peligro y debes mantener tu verdadera identidad en secreto.', 'Decides vender la información del científico a la corporación a cambio de una generosa recompensa. Esto te permite vivir una vida cómoda y segura, pero también te sume en la culpa por haber traicionado a alguien que confió en ti.']),
  'q2':
    ('Después de tomar la decisión de colaborar con la unidad especial de investigación de NeoTech, te diriges a la sede central de la unidad.',
    ['Una vez dentro, te reúnes con tu contacto en la unidad, la agente Sarah Ramirez. Ella es una experta en tecnología y te informa sobre los avances más recientes en la investigación. Optas por trabajar de manera estrecha con Sarah y utilizar las herramientas avanzadas de la unidad para buscar pruebas en la escena del crimen.', 'Decides mantener cierta distancia de la unidad y seguir tu propio enfoque investigativo. A pesar de colaborar, tienes dudas sobre las motivaciones ocultas del gobierno y prefieres no confiar completamente en ellos. Te sumerges en la base de datos central de NeoTech para obtener información adicional antes de dirigirte a la escena del crimen por tu cuenta.']),
  'q5':
    ('Después de colaborar estrechamente con la agente Sarah Ramirez y utilizar las herramientas avanzadas de la unidad para buscar pruebas en la escena del crimen, descubres pistas electrónicas intrigantes. Parece que el crimen está vinculado a una conspiración tecnológica en NeoTech. Sarah ha logrado descifrar un mensaje cifrado en un dispositivo cibernético encontrado en la escena del crimen.',
    ['Decides seguir la pista del mensaje cifrado y rastrear a los hackers detrás de la conspiración tecnológica. Sarah te proporciona una lista de contactos en el mundo underground de NeoTech que podrían tener información crucial. Optas por sumergirte en el mundo de los hackers y los ciberdelincuentes para obtener respuestas.', 'A pesar de la pista del mensaje cifrado, decides que es crucial investigar más a fondo a las corporaciones tecnológicas de NeoTech. Sospechas que las grandes empresas tienen un papel en esta conspiración y podrían estar encubriendo evidencia. Optas por infiltrarte en las oficinas centrales de una de las corporaciones tecnológicas más influyentes de la ciudad.']),
  'q11':
    ('Después de decidir seguir la pista del mensaje cifrado y rastrear a los hackers detrás de la conspiración tecnológica, te adentras en el mundo cibernético de NeoTech. Te encuentras con un grupo de hackers conocidos como "Los Cerebros Libres", quienes podrían tener información sobre la conspiración.',
    ['Decides unirte a "Los Cerebros Libres" y trabajar en estrecha colaboración con ellos para desentrañar la verdad detrás de la conspiración tecnológica. Formas parte de su equipo de hackers y te embarcas en misiones cibernéticas para descubrir pistas y exponer a las fuerzas oscuras detrás de NeoTech.', 'Prefieres mantener cierta distancia de "Los Cerebros Libres" y trabajar de manera independiente. Aunque estás dispuesto a colaborar, tienes dudas sobre sus motivaciones y no estás seguro de poder confiar plenamente en ellos. Optas por seguir tus propias pistas y realizar investigaciones paralelas a su trabajo.']),
  'q23':
    ('Después de unirte a "Los Cerebros Libres" y trabajar en estrecha colaboración con ellos para desentrañar la verdad detrás de la conspiración tecnológica, has estado involucrado en misiones cibernéticas peligrosas y has descubierto que la conspiración es más grande de lo que imaginabas. Ahora, has llegado a una encrucijada crítica.',
    ['Decides llevar la lucha directamente a las altas esferas de NeoTech. Te infiltras en la sede central de la megacorporación que está detrás de la conspiración y expones públicamente sus actividades corruptas. Esto desencadena un enfrentamiento directo entre "Los Cerebros Libres" y la megacorporación.', 'En lugar de una confrontación abierta, decides adoptar un enfoque más sigiloso y reunir pruebas sólidas antes de actuar. Utilizas las habilidades de hacking de "Los Cerebros Libres" para recopilar evidencia incriminatoria contra la megacorporación y exponer sus actividades ante las autoridades y la opinión pública.']),
  'q24':
    ('Después de decidir recopilar evidencia incriminatoria contra la conspiración tecnológica en NeoTech antes de actuar, te has sumergido aún más en el mundo de la ciberintriga. Has acumulado una gran cantidad de pruebas que implican a figuras poderosas en la conspiración.',
    ['Decides presentar tus pruebas a las autoridades de NeoTech y al público en general de manera abierta y directa. Revelas la verdad detrás de la conspiración tecnológica, exponiendo a las figuras corruptas en la ciudad ante la justicia y la opinión pública.', 'Optas por un enfoque más sutil y decides utilizar tus pruebas para extorsionar a las figuras corruptas y obligarlas a desmantelar la conspiración desde dentro. Negocias con las personas clave en la conspiración y las chantajeas para que tomen medidas para detener sus actividades ilegales.']),
  'q12':
    ('Después de decidir investigar más a fondo a las corporaciones tecnológicas de NeoTech, te preparas para infiltrarte en una de las corporaciones más influyentes de la ciudad, TechCorp. Has obtenido información que sugiere que esta corporación podría estar involucrada en la conspiración tecnológica.',
    ['Optas por infiltrarte en las oficinas centrales de TechCorp de manera sigilosa. Utilizas tus habilidades como detective para sortear la seguridad avanzada de la empresa y acceder a los archivos internos. Mientras exploras las instalaciones, descubres documentos y comunicaciones que vinculan a TechCorp con la conspiración.', 'En lugar de infiltrarte directamente, decides tomar un enfoque más indirecto. Optas por investigar a los empleados de TechCorp y buscar pistas sobre la conspiración a través de la información que puedas obtener de ellos. Te haces pasar por un periodista interesado en la empresa y te acercas a algunos de los empleados clave.']),
  'q25':
    ('Después de infiltrarte sigilosamente en las oficinas centrales de TechCorp y exponer públicamente sus actividades corruptas, te has convertido en un objetivo de la corporación. Han surgido revelaciones impactantes que han sacudido a NeoTech y han llevado a una confrontación directa con TechCorp.',
    ['Decides mantener la presión sobre TechCorp y enfrentarte a ellos directamente. Te unes a manifestaciones y protestas públicas en contra de la corporación, utilizando la opinión pública como un arma para desmantelar la influencia de TechCorp en la ciudad.', 'Prefieres adoptar un enfoque más discreto y sigiloso. Con la información que has reunido, trabajas en las sombras para socavar la influencia de TechCorp desde dentro. Colaboras con empleados descontentos de la corporación y filtradores de información para exponer sus secretos y debilitar gradualmente su poder.']),
  'q26':
    ('Después de decidir adoptar un enfoque más indirecto y buscar apoyo entre los disidentes políticos y los grupos de resistencia, has establecido una red clandestina de aliados que se oponen a la figura política corrupta en NeoTech. La conspiración se ha vuelto más peligrosa que nunca, y debes decidir cómo continuar tu lucha.',
    ['Decides liderar una revuelta abierta junto con tus aliados para derrocar a la figura política corrupta. Te unes a manifestaciones masivas y movimientos de resistencia que buscan expulsar al corrupto líder del poder.', 'Prefieres continuar operando en las sombras y trabajar en una estrategia de golpe silencioso. Tus aliados y tú planean un movimiento sigiloso para desenmascarar a la figura corrupta y exponer sus crímenes ante las autoridades y la opinión pública de manera coordinada.']),
  'q6':
    ('Después de decidir mantener cierta distancia de la unidad y seguir tu propio enfoque investigativo, te sumerges en la base de datos central de NeoTech para obtener información adicional. Encuentras registros que sugieren una posible conexión entre el crimen y una conspiración gubernamental oculta. La unidad especial de investigación parece estar involucrada en actividades sospechosas.',
    ['Decides confrontar a la agente Sarah Ramirez y cuestionar su lealtad a la unidad especial de investigación. Sospechas que podría estar involucrada en la conspiración y necesitas respuestas.', 'Prefieres mantener un perfil bajo y evitar confrontaciones internas. En lugar de confrontar a Sarah, decides investigar más a fondo las actividades de la unidad y la conspiración gubernamental por tu cuenta. Te infiltras en los archivos más confidenciales de la unidad en busca de pruebas sólidas.']),
  'q13':
    ('Después de decidir confrontar a la agente Sarah Ramirez y cuestionar su lealtad a la unidad especial de investigación, descubres que ella ha estado ocultando información sobre la conspiración gubernamental. Sarah finalmente admite que tiene vínculos con una facción secreta dentro del gobierno que está detrás de la conspiración.',
    ['Decides unirte a Sarah y trabajar en conjunto para exponer a la facción secreta dentro del gobierno. Ella te proporciona información privilegiada y acceso a recursos de la unidad especial de investigación que te ayudarán en tu lucha contra los conspiradores. Juntos, emprenden una misión peligrosa para revelar la verdad y detener a la facción secreta.', 'No puedes confiar plenamente en Sarah y prefieres mantener cierta distancia de la unidad especial de investigación. A pesar de la información que ella proporciona, decides llevar a cabo tu propia investigación paralela para exponer a la facción secreta del gobierno. Te sumerges en las profundidades de la política de NeoTech y buscas aliados entre los disidentes y los políticos descontentos.']),
  'q27':
    ('Después de unirte a la agente Sarah Ramirez y trabajar juntos para exponer a la facción secreta dentro del gobierno, te encuentras en medio de una confrontación directa con estas fuerzas corruptas. Has acumulado pruebas sólidas que implican a altos funcionarios del gobierno en la conspiración.',
    ['Decides llevar la lucha directamente a la opinión pública y las autoridades. Revelas las pruebas de la conspiración a los medios de comunicación y presentas una denuncia formal ante el gobierno.', 'Prefieres utilizar tus pruebas como palanca para negociar con la facción secreta dentro del gobierno. Te acercas a los conspiradores y les propones un acuerdo en el que retirarás las pruebas a cambio de que abandonen sus planes corruptos y se retiren de la vida pública.']),
  'q28':
    ('Después de optar por utilizar tus pruebas como palanca para negociar con la facción secreta dentro del gobierno, te has embarcado en un juego de intrigas políticas y negociaciones secretas. La facción corrupta se muestra dispuesta a hablar, pero sus verdaderas intenciones siguen siendo un misterio.',
    ['Decides seguir negociando con la facción secreta y llegar a un acuerdo con ellos para detener sus actividades corruptas y retirarse de la vida pública. En esta opción, trabajas en conjunto con la facción para encubrir la verdad y mantener sus secretos a salvo, a cambio de su promesa de cambiar su comportamiento y abandonar sus planes malignos.', 'No confías en las promesas de la facción secreta y decides exponer sus actividades corruptas de todas formas. Rompes las negociaciones y entregas tus pruebas a las autoridades y a los medios de comunicación, sin importar las consecuencias.']),
  'q14':
    ('Después de decidir investigar más a fondo las actividades de la unidad especial de investigación y la conspiración gubernamental por tu cuenta, te sumerges en una red de engaños y secretos políticos en NeoTech. Has descubierto pistas que sugieren que la conspiración está conectada con una figura política de alto rango en la ciudad.',
    ['Decides centrarte en la figura política sospechosa y te infiltras en su círculo cercano para obtener información crucial. Te haces pasar por un asesor político y te acercas a la persona en cuestión, buscando pistas sobre sus actividades ilegales y su participación en la conspiración.', 'Prefieres buscar apoyo entre los disidentes políticos y los grupos de resistencia que se oponen a la figura política sospechosa. Te unes a un grupo clandestino de opositores y trabajas con ellos para exponer la verdad y derrocar a la figura corrupta desde las sombras.']),
  'q29':
    ('Después de liderar una revuelta abierta junto con tus aliados para derrocar a la figura política corrupta en NeoTech, te encuentras en medio de un enfrentamiento directo con las fuerzas gubernamentales corruptas. La ciudad está en agitación y debes decidir cómo manejar la situación.',
    ['Decides mantener la presión sobre las fuerzas gubernamentales corruptas y continuar liderando la resistencia abierta. Te unes a las protestas y manifestaciones, manteniendo la lucha en las calles y tratando de expulsar completamente a la figura corrupta y su régimen del poder. ', 'Optas por un enfoque más pragmático y buscas una solución negociada. A pesar de las diferencias, te acercas a las fuerzas gubernamentales corruptas con la intención de llegar a un acuerdo que permita una transición de poder pacífica. Trabajas en un proceso de reconciliación y cambio político que permita a la ciudad sanar y avanzar.']),
  'q30':
    ('Después de optar por un enfoque más discreto y trabajar en las sombras para socavar la influencia de la figura política corrupta en NeoTech, te has convertido en una fuerza clandestina que opera en busca de un cambio político. La conspiración política se ha vuelto más peligrosa que nunca, y debes decidir cómo continuar tu lucha.',
    ['Decides seguir operando en las sombras y aumentar la presión sobre la figura política corrupta y sus seguidores. Realizas una serie de acciones encubiertas para exponer aún más sus actividades ilícitas, socavando su poder desde dentro.', 'Optas por una estrategia más audaz y decides revelar públicamente tus pruebas y exponer a la figura política corrupta ante la opinión pública y las autoridades. Utilizas las pruebas que has acumulado durante tu tiempo en las sombras para desmantelar su influencia de manera abierta y directa.']),
  'q31':
    ('Olvidaste tu cometido inicial, ahora eres un renegado. Te dejaste llevar por sus ideales y ahora eres libre de moverte por toda la ciudad, incluso por sus más bajos barrios. Eres más libre que nunca, aunque el culpable del crimen es tan libre como tú... Incluso podría ser alguno de tus nuevos colegas. GAME OVER',
    ['', '']),
  'q32':
    ('Con la información proporcionada por el informante, logras dar con el asesino. Fue un enfrentamiento difícil, sólo tú saliste con vida. VICTORIA',
    ['', '']),
  'q33':
    ('La base de los renegados estaba altamente vigilada. Entrar fue algo realmente difícil y salir no será posible. Te has encontrado con mucha más resistencia de la que esperabas y moriste a manos de varios renegados. GAME OVER',
    ['', '']),
  'q34':
    ('La presión del gobierno fue demasiado fuerte. El criminal sigue libre, pero prefieres guardar tu vida para colaborar en otros casos que no supongan inmiscuirse en asuntos del gobierno corrupto de NeoTech.',
    ['', '']),
  'q35':
    ('Con la información proporcionada, logras dar con el asesino. Fue un enfrentamiento difícil, sólo tú saliste con vida. Además, ahora con cuentas valiosa información del bajo mundo que servirá para futuros casos. VICTORIA',
    ['', '']),
  'q36':
    ('Te desenfocaste por completo. Tu instinto de detective te dice que hay casos más importantes que resolver con la nueva información que cuentas. El criminal sigue libre, pero ahora no lo consideras un problema tuyo',
    ['', '']),
  'q37':
    ('Vives tranquilo con el dinero. Siempre soñaste con poder retirarte de tan agitado mundo. Ahora que tienes todo cumplir tus sueños, los criminales y cualquier otro caso que antes te preocupaba ya no suponen un problema. GAME OVER.',
    ['', '']),
  'q38':
    ('Después de optar por presentar tus pruebas a las autoridades de NeoTech y al público en general de manera abierta y directa, has desencadenado una serie de eventos que afectarán el destino de la ciudad y el desenlace de tu lucha contra la conspiración tecnológica. Tu valentía al revelar la verdad ante la opinión pública y las autoridades ha tenido un impacto abrumador. Las pruebas que presentaste son contundentes y, como resultado, las figuras corruptas en NeoTech son arrestadas y llevadas ante la justicia. La ciudadanía y los medios de comunicación celebran tu papel en la exposición de la corrupción y en la lucha por la justicia. La megacorporación NeoTech se tambalea bajo la presión pública y legal. Se inician investigaciones exhaustivas y reformas para garantizar que la corrupción no vuelva a tener cabida en la ciudad. Tu valentía te convierte en un héroe en la lucha por la integridad y la transparencia en NeoTech.',
    ['', '']),
  'q49':
    ('Después de optar por un enfoque más sigiloso y trabajar para exponer las actividades corruptas de la megacorporación NeoTech, tus acciones han desencadenado una serie de eventos que afectarán el destino de la ciudad y de "Los Cerebros Libres". Tu astucia y habilidades de exposición han tenido un impacto profundo en la lucha contra la megacorporación NeoTech. Al exponer sus actividades corruptas ante las autoridades y la opinión pública, se desencadena una serie de investigaciones y acciones legales contra la corporación y sus líderes. La megacorporación se tambalea bajo la presión pública y legal. Tus aliados de "Los Cerebros Libres" reconocen tu contribución invaluable en la lucha y te eligen como su líder. Juntos, trabajan para promover la transparencia y la responsabilidad en NeoTech, y se convierten en una fuerza poderosa para el cambio. La ciudad experimenta una transformación significativa hacia un futuro más ético y equitativo.',
    ['', '']),
  'q40':
    ('Después de liderar una revuelta abierta junto con tus aliados para derrocar a la figura política corrupta en NeoTech, la ciudad se encuentra en un estado de agitación y cambio. Tu valentía ha tenido un impacto profundo en la transformación de la ciudad, pero los desafíos persisten. Tu liderazgo en la revuelta abierta ha sido un éxito rotundo. La figura política corrupta es destituida de su cargo y se inicia un proceso de transición hacia un gobierno más democrático y transparente en NeoTech. La ciudadanía celebra tu papel en la lucha por la justicia y la democracia, y eres considerado un héroe. A medida que la ciudad se estabiliza, participas activamente en la formación del nuevo gobierno. Trabajas junto con otros líderes de la resistencia y activistas para establecer un sistema político que garantice la rendición de cuentas y la igualdad de oportunidades para todos los ciudadanos. NeoTech comienza a recuperarse de los años de corrupción y abuso de poder.',
    ['', '']),
  'q41':
    ('La facción secreta acepta tu solicitud. Te reúnes con ellos en uno de los barrios bajos de NeoTech. Rápidamente descubres que todo era una trampa y alcanzas a huir por poco del lugar. Estás en una encrucijada, debes elegir rápido:',
    ['Refugiarte en un lugar seguro y salir del foco de atención.', 'Liderar una trifulca abierta en contra del gobierno solicitando la protección del pueblo']),
  'q42':
    ('Después de decidir utilizar tus pruebas para exponer públicamente las actividades corruptas de la conspiración tecnológica en NeoTech, has enfrentado directamente a las figuras políticas corruptas y has desencadenado un enfrentamiento abierto en la ciudad. Tu valentía y determinación para exponer la corrupción han tenido un impacto abrumador en NeoTech. Las pruebas que proporcionaste son irrefutables y han llevado a la detención de las figuras políticas corruptas involucradas en la conspiración. La ciudad celebra tu valiente acto y eres considerado un héroe que luchó por la justicia y la transparencia en un mundo donde la corrupción estaba profundamente arraigada. Las reformas políticas se implementan rápidamente, y NeoTech avanza hacia un futuro más democrático y honesto.',
    ['', '']),
  'q43':
    ('Confiaste demasiado en el gobierno corrupto, que hace parte de la facción secreta y has sido traicionado, en mitad de la noche mientras disfrutabas de una cena, feliz por todas las negociaciones que habías logrado. Entra a tu casa sigilosamente un asesino veterano controlado por el gobierno y acaba contigo sin dejar ningún rastro. GAME OVER',
    ['', '']),
  'q44':
    ('Después de liderar una revuelta abierta y exponer públicamente a la figura política corrupta en NeoTech, la ciudad se encuentra en un estado de agitación y cambio. Las fuerzas gubernamentales corruptas han sido debilitadas, pero aún existe incertidumbre sobre el futuro de la ciudad. Tu papel como líder de la resistencia ha sido fundamental en esta transformación, y debes decidir cómo continuar y consolidar el cambio.',
    ['Decides participar activamente en la formación de un nuevo gobierno democrático en NeoTech. Trabajas con otros líderes de la resistencia y activistas para organizar elecciones libres y justas que permitan a la ciudadanía elegir a sus representantes. Esta elección te lleva por un camino de reconstrucción política y transición hacia un gobierno más transparente y justo. Sin embargo, enfrentarás desafíos en la estabilización y unificación de la ciudad tras los eventos tumultuosos.', 'Prefieres retirarte de la vida pública y buscar una vida más tranquila después de la agitación. Aunque has desempeñado un papel crucial en la transformación de NeoTech, decides que es hora de alejarte de la política y vivir una vida más tranquila.']),
  'q45':
    ('Decidiste tomar un puesto político importante para vigilar de cerca la consolidación del cambio en la ciudad. Aunque es tan fácil como parece, pues aun tienes que lidiar con diferentes miembros de la oposición, que permanecen en férrreos en sus puestos. Estás ante una decisión muy importante:',
    ['Decides seguir negociando con el gobierno y llegar a un acuerdo con ellos para detener sus actividades corruptas y retirarte de la vida pública.Trabajas  en conjunto con la facción para encubrir la verdad y mantener algunos secretos a salvo, a cambio de su promesa de cambiar su comportamiento y abandonar sus planes malignos.', 'Prefieres retirarte de la vida pública y buscar una vida más tranquila después de la agitación. Aunque has desempeñado un papel crucial en la transformación de NeoTech, decides que es hora de alejarte de la política y vivir una vida más tranquila.']),
  'q46':
    ('Olvidaste tu cometido inicial, aunque la ciudad ahora es más segura, el culpable del crimen aún deambula por ella a sus anchas. GAME OVER',
    ['', '']),
  'q47':
    ('Después de optar por seguir operando en las sombras y aumentar la presión sobre la figura política corrupta en NeoTech, has continuado desmantelando su influencia de manera encubierta. Has realizado una serie de acciones sigilosas para exponer aún más sus actividades ilícitas, socavando su poder desde dentro. La figura corrupta ha quedado prácticamente despojada de su autoridad y sus seguidores han perdido la confianza en ella. La estrategia de operar en las sombras ha dado sus frutos, y has logrado una victoria significativa al socavar y debilitar con éxito la influencia de la figura política corrupta en NeoTech. La ciudad ha comenzado a sanar y se está moviendo hacia un futuro más transparente y justo. Aunque tu lucha ha sido discreta, ha tenido un impacto duradero en la transformación de la ciudad. Esta es una victoria en tu búsqueda por la justicia y la reforma en NeoTech. El detective Alex Mercer ha triunfado en su lucha contra la corrupción política y ha contribuido al cambio positivo en la ciudad. Tu habilidad para operar en las sombras y socavar al enemigo ha sido fundamental en esta victoria.',
    ['', ''])
}

In [ ]:
#Personalization of names
def personalization(fragment,input_name,output_name):
  ex= r".*"+input_name+".*"
  pattern = re.compile(ex)

  result=fragment
  if bool(pattern.match(fragment)):
    result = re.sub(input_name,output_name,fragment)

  return result

In [ ]:
transducer = FST()
transducer.add_transitions([

    ('q0','H','q1',['']),
    ('q1','a','q2',['']),
    ('q2','b','q3',['']),
    ('q3','l','q4',['']),
    ('q4','a','q5',['']),
    ('q5','r','q6',['']),

    ('q6',' ','q7',['']),

    ('q7','c','q8',['']),
    ('q8','o','q9',['']),
    ('q9','n','q10',['']),

    ('q10',' ','q11',['']),

    ('q11','a','q12',['']),
    ('q12','l','q13',['']),
    ('q13','g','q14',['']),
    ('q14','u','q15',['']),
    ('q15','i','q16',['']),
    ('q16','e','q17',['']),
    ('q17','n','q18',['1']),

    ('q0','S','q19',['']),
    ('q19','e','q20',['']),
    ('q20','l','q21',['']),
    ('q21','e','q22',['']),
    ('q22','c','q23',['']),
    ('q23','c','q24',['']),
    ('q24','i','q25',['']),
    ('q25','o','q26',['']),
    ('q26','n','q27',['']),
    ('q27','a','q28',['']),
    ('q28','r','q29',['']),

    ('q29',' ','q30',['']),

    ('q30','o','q31',['']),
    ('q31','p','q32',['']),
    ('q32','c','q33',['']),
    ('q33','i','q34',['']),
    ('q34','o','q35',['']),
    ('q35','n','q36',['']),

    ('q36',' ','q37',['']),
    ('q36',' ','q39',['']),

    ('q37','1','q18',['2']),
    ('q39','2','q18',['3']),

    ('q0','I','q40',['']),
    ('q40','n','q41',['']),
    ('q41','i','q42',['']),
    ('q42','c','q43',['']),
    ('q43','i','q44',['']),
    ('q44','a','q45',['']),
    ('q45','r','q46',['']),

    ('q46',' ','q47',['']),

    ('q47','j','q48',['']),
    ('q48','u','q49',['']),
    ('q49','e','q50',['']),
    ('q50','g','q51',['']),
    ('q51','o','q18',['4']),

    ('q0','P','q52',['']),
    ('q52','e','q53',['']),
    ('q53','r','q54',['']),
    ('q54','s','q55',['']),
    ('q55','o','q56',['']),
    ('q56','n','q57',['']),
    ('q57','a','q58',['']),
    ('q58','l','q59',['']),
    ('q59','i','q60',['']),
    ('q60','z','q61',['']),
    ('q61','a','q62',['']),
    ('q62','r','q63',['']),

    ('q63',' ','q64',['']),

    ('q64','n','q65',['']),
    ('q65','o','q66',['']),
    ('q66','m','q67',['']),
    ('q67','b','q68',['']),
    ('q68','r','q69',['']),
    ('q69','e','q18',['5']),


    ('q0','V','q70',['']),
    ('q70','o','q71',['']),
    ('q71','l','q72',['']),
    ('q72','v','q73',['']),
    ('q73','e','q74',['']),
    ('q74','r','q75',['']),

    ('q75',' ','q76',['']),

    ('q76','a','q77',['']),

    ('q77',' ','q78',['']),

    ('q78','j','q79',['']),
    ('q79','u','q80',['']),
    ('q80','g','q81',['']),
    ('q81','a','q82',['']),
    ('q82','r','q18',['6']),

    ('q0','F','q83',['']),
    ('q83','i','q84',['']),
    ('q84','n','q85',['']),
    ('q85','a','q86',['']),
    ('q86','l','q87',['']),
    ('q87','i','q88',['']),
    ('q88','z','q89',['']),
    ('q89','a','q90',['']),
    ('q90','r','q91',['']),

    ('q91',' ','q92',['']),

    ('q92','j','q93',['']),
    ('q93','u','q94',['']),
    ('q94','e','q95',['']),
    ('q95','g','q96',['']),
    ('q96','o','q18',['7'])

])

transducer.add_start_state('q0')
transducer.add_final_state('q18')

def inputVerification(input):
  try:
    result = "".join(list(transducer.translate(input))[0])
  except:
    return 0
  return int(result)


In [ ]:
#Automata

q0 = State("q0")
q1 = State("q1")
q2 = State("q2")
q3 = State("q3")
q4 = State("q4")
q5 = State("q5")
q6 = State("q6")
q7 = State("q7")
q8 = State("q8")
q9 = State("q9")
q10 = State("q10")
q11 = State("q11")
q12 = State("q12")
q13 = State("q13")
q14 = State("q14")
q15 = State("q15")
q16 = State("q16")
q17 = State("q17")
q18 = State("q18")
q19 = State("q19")
q20 = State("q20")
q21 = State("q21")
q22 = State("q22")
q23 = State("q23")
q24 = State("q24")
q25 = State("q25")
q26 = State("q26")
q27 = State("q27")
q28 = State("28")
q29 = State("q29")
q30 = State("q30")
q31 = State("q31")
q32 = State("q32")
q33 = State("q33")
q34 = State("q34")
q35 = State("q35")
q36 = State("q36")
q37 = State("q37")
q38 = State("q38")
q39 = State("q39")
q40 = State("q40")
q41 = State("q41")
q42 = State("q42")
q43 = State("q43")
q44 = State("q44")
q45 = State("q45")
q46 = State("q46")
q47 = State("q47")

dfa = DeterministicFiniteAutomaton(
states={q0, q1, q2, q3, q4, q5, q6, q7},
input_symbols={"1", "2","3"},
start_state=q0,
final_states={q31,q32,q33,q34,q35,q36,q37,q38,q39,q40,q42,q43,q46,q47} # Define the set of accepting states
)

dfa.add_transitions([

(q0,"1",q1),
(q0,"2",q1),

(q1,"1",q3),
(q1,"2",q4),

(q2,"1",q5),
(q2,"2",q6),

(q3,"1",q7),
(q3,"2",q9),

(q7,"1",q15),
(q7,"2",q16),

(q8,"1",q17),
(q8,"2",q18),

(q4,"1",q9),
(q4,"2",q10),

(q9,"1",q19),
(q9,"2",q20),

(q10,"1",q21),
(q10,"2",q22),

(q5,"1",q11),
(q5,"2",q12),

(q11,"1",q23),
(q11,"2",q24),

(q12,"1",q25),
(q12,"2",q26),

(q6,"1",q13),
(q6,"2",q14),

(q13,"1",q27),
(q13,"2",q28),

(q14,"1",q29),
(q14,"2",q30),

(q15,"1",q27),
(q15,"2",q17),

(q16,"1",q31),
(q16,"2",q17),

(q17,"1",q32),
(q17,"2",q33),

(q18,"1",q32),
(q18,"2",q26),

(q19,"1",q34),
(q19,"2",q43),

(q20,"1",q35),
(q20,"2",q30),

(q21,"1",q36),

(q22,"1",q35),
(q22,"2",q37),

(q23,"1",q38),
(q23,"2",q39),

(q24,"1",q38),
(q24,"2",q39),

(q25,"1",q40),
(q25,"2",q47),

(q26,"1",q44),
(q26,"2",q47),

(q27,"1",q41),
(q27,"2",q44),

(q28,"1",q42),
(q28,"2",q43),

(q29,"1",q43),
(q29,"2",q44),

(q30,"1",q44),
(q30,"2",q47),

(q41,"1",q44),

(q44,"1",q45),
(q44,"2",q46),

(q45,"1",q46),
])

In [ ]:
gramatica = {
    'COMENTARIO': [
        'PERSONAJE1 : LINEACOMENTARIO PERSONAJE2',
        'PERSONAJE1 : LINEACOMENTARIO',
    ],
    'LINEACOMENTARIO': [
        'Hola, ¿cómo estás?',
        '¿Qué tal estás en este mundo futurista?',
        'Buenos días en NeoTech',
        'Buenas tardes, detective. ¿Cómo va la investigación?',
        '¿Cómo va todo en esta ciudad tecnológica?',
        '¿Qué tal tu día como detective en este mundo avanzado?',
        '¿Qué novedades hay en la investigación del crimen?',
        '¿En qué andas investigando?',
        '¿Cómo te sientes con el avance de la investigación?',
        '¿Qué haces por aquí en NeoTech?',
        'Un placer verte, detective',
        'Hola, ¿cómo ha ido tu día resolviendo crímenes?',
        '¿Qué tal la jornada en la ciudad del futuro?',
        '¿Cómo estuvo tu día de detective?',
        '¿Tienes planes para descubrir la verdad?',
        '¡Hola! ¿En qué parte de NeoTech estás ahora?',
        '¿Qué cuentas sobre el caso que estás investigando?',
        '¿Cuál es la novedad en tu investigación?',
        '¿Alguna pista sobre el crimen en este mundo futurista?',
        '¿Qué hay de nuevo en tu búsqueda de la verdad?',
        '¿Cómo has estado investigando?',
        '¿Algún avance en tu búsqueda de justicia?',
        '¿En qué estás pensando respecto al caso?',
        '¿Qué estás haciendo para resolver este misterio?',
        '¿Has encontrado alguna pista importante?',
        '¿Cómo va todo en este mundo lleno de tecnología?'
    ],
    'RESPUESTA': [
        'LINEARESPUESTA',
        'LINEARESPUESTA LINEACOMENTARIO',
    ],
    'LINEARESPUESTA': [
        'Estoy bien, gracias por preguntar.',
        'La investigación avanza poco a poco, pero va bien.',
        'Navegando por este mundo futurista en busca de respuestas.',
        'Todo en orden, tratando de desentrañar este misterio.',
        'Estoy intrigado por las posibilidades que ofrece esta ciudad avanzada.',
        'Analizando los datos que tenemos hasta ahora.',
        'Las tecnologías de NeoTech hacen la investigación más interesante.',
        'Investigando con determinación para resolver este caso.',
        'Descubriendo secretos en cada rincón de esta ciudad futurista.',
        'Tratando de conectar los puntos para entender la verdad.',
        'Cada día es un nuevo reto en este mundo tecnológico.',
        'Analizando las pruebas que encontré en la última escena del crimen.',
        'Listo para descubrir la verdad oculta en esta ciudad avanzada.',
        'Explorando NeoTech en busca de pistas.',
        'Enfocado en resolver este caso y llevar a los culpables ante la justicia.',
        'Buscando respuestas en esta ciudad llena de avances tecnológicos.',
        'Listo para enfrentar los desafíos que este caso presenta.',
        'Investigando incansablemente para resolver este crimen en NeoTech.',
        'Trabajando duro para encontrar la verdad y cerrar este caso.',
        'Cada pista me acerca más a la resolución del crimen.',
        'Investigando de manera meticulosa para no dejar ningún cabo suelto.',
        'Siguiendo las pistas y buscando conexiones en este mundo futurista.',
        'Concentrado en descubrir la verdad detrás de este enigma tecnológico.',
        'Listo para enfrentar lo que sea necesario para resolver este caso.'
    ],
    'PERSONAJE1': ['Detective Alex Mercer'],
    'PERSONAJE2': ['Informante', 'Testigo', 'Sospechoso', 'Colega']
}

def generar_oracion(simbolo_inicial):
    if simbolo_inicial not in gramatica:
        return simbolo_inicial

    produccion_elegida = random.choice(gramatica[simbolo_inicial])

    oracion_generada = ' '.join([generar_oracion(simbolo) for simbolo in produccion_elegida.split()])
    return oracion_generada

# Generar un comentario
print("Comentario generado:")
comentario_generado = generar_oracion('COMENTARIO')
print(comentario_generado)

# Generar una respuesta
print("\nRespuesta generada:")
respuesta_generada = generar_oracion('RESPUESTA')
print(respuesta_generada)


In [ ]:
criticas_personajes = [
    "Sarah siempre se ha creído la más lista de la sala. Pero a veces su obsesión por demostrarlo la lleva a pasarse por alto detalles cruciales. No todo es código y algoritmos, a veces se necesita sentido común y experiencia.",
    "Alex es un buen detective, eso no lo niego. Pero a veces se aferra tanto a sus métodos tradicionales que se pierde las oportunidades que la tecnología podría ofrecer. Si no se actualiza, se quedará atrás en este mundo en constante evolución.",
    "TechCorp, la típica corporación que piensa que puede comprar su camino hacia cualquier cosa. La verdad está ahí, enterrada bajo capas de secretos y avaricia. Solo espero que la justicia prevalezca y que no logren comprarla también.",
    "NeoTech, la ciudad que se enorgullece de su progreso tecnológico. Pero detrás de las luces brillantes y los avances, se esconden oscuros secretos. La tecnología no lo es todo; a veces, lo humano se pierde en esta búsqueda de perfección mecánica.",
    "Los hackers, creyendo que son los héroes de la era digital. Pero detrás de sus máscaras y líneas de código, a menudo se esconden individuos impulsados por motivos egoístas. No todos merecen ser glorificados, algunos solo quieren ver arder el mundo."
]

In [ ]:
class TestPersonalizationMethod(unittest.TestCase):

    def test_Personalization(self):
        sentence="Juan estaba jugando futbol, mientras Pedro y Gabriel juegan baloncesto, y Adriana nada"
        self.assertEqual(personalization(sentence,"Pedro","Alberto"), "Juan estaba jugando futbol, mientras Alberto y Gabriel juegan baloncesto, y Adriana nada")
        self.assertEqual(personalization(sentence,"Juan","Miguel"), "Miguel estaba jugando futbol, mientras Pedro y Gabriel juegan baloncesto, y Adriana nada")
        self.assertEqual(personalization(sentence,"Gabriel","Mateo"), "Juan estaba jugando futbol, mientras Pedro y Mateo juegan baloncesto, y Adriana nada")
        self.assertEqual(personalization(sentence,"Adriana","Lucia"), "Juan estaba jugando futbol, mientras Pedro y Mateo juegan baloncesto, y Adriana nada")

In [ ]:

class TestVeficationMethod(unittest.TestCase):

    def test_inputVerification(self):
        self.assertEqual(inputVerification('Hablar con alguien'), 1)
        self.assertEqual(inputVerification('Seleccionar opcion 1'), 2)
        self.assertEqual(inputVerification('Seleccionar opcion 2'), 3)
        self.assertEqual(inputVerification('Iniciar juego'), 4)
        self.assertEqual(inputVerification('Personalizar nombre'), 5)
        self.assertEqual(inputVerification('Volver a jugar'), 6)
        self.assertEqual(inputVerification('Finalizar juego'), 7)
        self.assertEqual(inputVerification('Buenos dias'), 0)
        self.assertEqual(inputVerification('Empezar juego'), 1)